In [1]:
import requests
import pandas as pd
from collections import defaultdict
from copy import deepcopy

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

LEAGUE_IDS = {
    2024: "1064047033306136576",
    2025: "1180226065723957248"
}

MAX_WEEKS = 18   # Regular season + playoffs (safe upper bound)


/Users/jose/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def get_league_settings(league_id):
    league = requests.get(
        f"https://api.sleeper.app/v1/league/{league_id}"
    ).json()

    return {
        "num_teams": league["total_rosters"],
        "draft_rounds": league["settings"]["draft_rounds"]
    }


In [3]:
def initialize_pick_state(num_teams, draft_rounds, seasons):
    """
    Each roster starts with 1 pick per round per season
    """
    state = defaultdict(lambda: defaultdict(int))

    for roster_id in range(1, num_teams + 1):
        for season in seasons:
            for rnd in range(1, draft_rounds + 1):
                key = f"{season}_R{rnd}"
                state[roster_id][key] += 1

    return state


In [4]:
def get_transactions(league_id, week):
    url = f"https://api.sleeper.app/v1/league/{league_id}/transactions/{week}"
    resp = requests.get(url)
    return resp.json() if resp.status_code == 200 else []


In [5]:
def apply_transactions(state, transactions):
    """
    Mutates state in-place
    """
    for txn in transactions:
        if txn["type"] != "trade":
            continue

        for pick in txn.get("draft_picks", []):
            season = pick["season"]
            rnd = pick["round"]
            new_owner = pick["roster_id"]
            prev_owner = pick["previous_owner_id"]

            key = f"{season}_R{rnd}"

            if prev_owner is not None:
                state[prev_owner][key] -= 1
            state[new_owner][key] += 1


In [6]:
def reconstruct_pick_history(league_id, season_year):
    settings = get_league_settings(league_id)
    num_teams = settings["num_teams"]
    draft_rounds = settings["draft_rounds"]

    # Track picks for this season + future seasons
    seasons_tracked = [season_year, season_year + 1, season_year + 2]

    state = initialize_pick_state(num_teams, draft_rounds, seasons_tracked)

    snapshots = []

    for week in range(1, MAX_WEEKS + 1):
        transactions = get_transactions(league_id, week)
        apply_transactions(state, transactions)

        snapshot = {
            "season": season_year,
            "week": week,
            "picks": deepcopy(state)
        }
        snapshots.append(snapshot)

    return snapshots


In [7]:
all_pick_history = {}

for season, league_id in LEAGUE_IDS.items():
    print(f"Reconstructing picks for {season}...")
    all_pick_history[season] = reconstruct_pick_history(league_id, season)


Reconstructing picks for 2024...
Reconstructing picks for 2025...


In [8]:
def flatten_pick_history_cumulative(history):
    rows = []

    for snap in history:
        season = snap["season"]
        week = snap["week"]

        for roster_id, picks in snap["picks"].items():
            # Total picks per pick_season & round
            pick_totals = defaultdict(int)
            for pick_key, count in picks.items():
                pick_season, rnd = pick_key.split("_R")
                pick_totals[(int(pick_season), int(rnd))] += count

            for (pick_season, rnd), total_count in pick_totals.items():
                rows.append({
                    "season": season,
                    "week": week,
                    "roster_id": roster_id,
                    "pick_season": pick_season,
                    "round": rnd,
                    "total_picks": total_count
                })

    return pd.DataFrame(rows)


In [9]:
df_2024 = flatten_pick_history_cumulative(all_pick_history[2024])
df_2025 = flatten_pick_history_cumulative(all_pick_history[2025])

In [10]:
df_2024.head()

,season,week,roster_id,pick_season,round,total_picks
0,2024,1,1,2024,1,0
1,2024,1,1,2024,2,-1
2,2024,1,1,2024,3,1
3,2024,1,1,2024,4,0
4,2024,1,1,2025,1,1


In [11]:
# def add_pick_metadata(df):
#     df = df.copy()
#     df["pick_season"] = df["pick"].str.split("_").str[0].astype(int)
#     df["round"] = df["pick"].str.split("_R").str[1].astype(int)
#     return df


In [12]:
# df_2024 = add_pick_metadata(df_2024)
# df_2025 = add_pick_metadata(df_2025)

In [13]:
def compute_total_picks_per_round(df):
    totals = (
        df
        .groupby(
            ["season", "week", "roster_id", "pick_season", "round"],
            as_index=False
        )["total_picks"]  # <- use 'total_picks' here
        .sum()
    )

    # Clamp negatives to zero (preseason artifact)
    totals["total_picks"] = totals["total_picks"].clip(lower=0)

    return totals


In [ ]:
df_2024_round_totals = compute_total_picks_per_round(df_2024)
df_2025_round_totals = compute_total_picks_per_round(df_2025)

In [15]:
df_2024_round_totals[
    (df_2024_round_totals["roster_id"] == 1) &
    (df_2024_round_totals["pick_season"] == 2025)
]

,season,week,roster_id,pick_season,round,total_picks
4,2024,1,1,2025,1,1
5,2024,1,1,2025,2,0
6,2024,1,1,2025,3,1
7,2024,1,1,2025,4,1
134,2024,2,1,2025,1,1
135,2024,2,1,2025,2,0
136,2024,2,1,2025,3,1
137,2024,2,1,2025,4,1
264,2024,3,1,2025,1,1
265,2024,3,1,2025,2,0


In [16]:
df_2024[df_2024['roster_id'] == 1]

,season,week,roster_id,pick_season,round,total_picks
0,2024,1,1,2024,1,0
1,2024,1,1,2024,2,-1
2,2024,1,1,2024,3,1
3,2024,1,1,2024,4,0
4,2024,1,1,2025,1,1
5,2024,1,1,2025,2,0
6,2024,1,1,2025,3,1
7,2024,1,1,2025,4,1
8,2024,1,1,2026,1,1
9,2024,1,1,2026,2,1
